In [1]:
import numpy as np
import random

#### Functions 

In [2]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    Returns:
    parameters: dictionary containing:
                        Wax - Weight matrix multiplying the input, shape (n_a, n_x)
                        Waa - Weight matrix multiplying the hidden state, shape (n_a, n_a)
                        Wya - Weight matrix (for the hidden-state to the output), shape (n_y, n_a)
                        b -  Bias,  (n_a, 1)
                        by - Bias relating the hidden-state to the output, shape (n_y, 1)
    """
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # 
    
    return a_next, p_t

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    # initialize loss to 0
    loss = 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0  => input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # one step forward 
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

def rnn_backward(X, Y, parameters, cache):
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    ### START CODE HERE ###
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    ### END CODE HERE ###
    
    return gradients, a
 

#### Dataset, preprocessing 

In [3]:
data=open('dinos.txt', 'r').read()

In [4]:
data=data.lower()
chars=list(set(data))

In [5]:
list(set(data))[:8]

['n', 'e', 'j', 'y', 'b', 'o', 'k', 'z']

In [6]:
data_size, vocab_size=len(data), len(chars)
print("there are {} characters and {} unique characters in the data".format(data_size, vocab_size))

there are 19909 characters and 27 unique characters in the data


index the characters

In [7]:
char_to_ix={a:b for b,a in enumerate(sorted(chars))}
ix_to_char={b:a for b,a in enumerate(sorted(chars))}
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### model 
- initialization of parameters
- forward propagation (compute loss function)
- backward propagation (compute the gradients with respect to the loss function)
- clip the gradients (avoid exploding gradients) - before updating the parameters. Simple version (range -10, 10 + truncate)
- update parameters with gradient descent
- return the learned parameters
<br><br>
X - a list of characters in training set

##### numpy.clip(a,a_min, a_max, out=None)
''' Clip (limit) the values in an array
out: the results will be placed in this array

In [8]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby=gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    
    for i in [dWaa, dWax, dWya, db, dby]:
        i=np.clip(i,-maxValue, maxValue, out=i)
        
    gradients={'dWaa':dWaa, 'dWax':dWax, 'dWya':dWya, 'db':db, 'dby':dby}
    return gradients

In [9]:
#test
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]


#### Sampling 
1. 'dummy' input x1=zeros and a0 =zeros <br>
2. forward prop. to get a1 an y_hat_1.
:

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)$$
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y$$
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })$$

3 Next character (according to the probability distribution $(\hat{y}^{\angle t+1 \rangle}$)


In [10]:
def  sample(parameters, char_to_ix):
    '''Arguments:
    parameters -  dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix - dictionary mapping each character to an index.
    
    Returns:
    indices - a list of length n containing'''
    
    # parameters and shapes: 
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size=by.shape[0]
    n_a=Waa.shape[1]
    
    # one-hot vector x for the 1.st character, just zeros
    x=np.zeros([vocab_size, 1])
    a_prev = np.zeros([n_a,1])
        
    indices = []
    idx=-1 # a flag to detect a newline char., initialize: -1
    
    # Loop over timesteps t (max 50):
    # At each timestep = the most probably char. (index) will be appended.
    
    counter=0
    newline=char_to_ix['\n']
    
    while(idx!=newline and counter!=50):
        a=np.tanh(np.dot(Wax,x)+np.dot(Waa,a_prev)+b)
        z=np.dot(Wya,a)+by
        y=softmax(z)
        
        idx=np.random.choice(range(len(y)), p=y.ravel())
        
        indices.append(idx)
        
        x=np.zeros([vocab_size,1])
        x[idx]=1
        
        a_prev=a
        
        counter+=1
        
    if counter==50:
        indices.append(char_to_ix['\n'])
    
    return indices

In [11]:
#test
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

indices=sample(parameters, char_to_ix)
print('Sampling: \nIndices: {}\nCharacters: {}'.format(indices, [ix_to_char[i] for i in indices]))

Sampling: 
Indices: [12, 23, 24, 14, 7, 2, 10, 23, 25, 12, 3, 24, 15, 24, 3, 20, 3, 17, 4, 0]
Characters: ['l', 'w', 'x', 'n', 'g', 'b', 'j', 'w', 'y', 'l', 'c', 'x', 'o', 'x', 'c', 't', 'c', 'q', 'd', '\n']


#### Gradient descent
- compute the loss
- compute gradients of the loss with respect to the parameters
- clip the gradients if necessary
- update parameters

In [12]:
# using the previously defined functions
def optimize(X,Y,a_prev, parameters, learning_rate=0.01):
    '''
    Runs  one step of the optimization to train the model
    
    arguments:
    X - list of integers, (they map to characters in the vocabulary).
    Y - list of integers
    a_prev - previous hidden state
    parameters -  dictionary of parameters (Wax, Waa, Wya, b, by)
    learning_rate -- learning rate for the model.
    Returns:
    loss (cross-entropy)
    gradients -  dictionary of gradients: dWax, dWaa, dWya, db, dby
    a[len(X)-1] -- the last hidden state
    '''
    #forward prop.
    loss, cache=rnn_forward(X,Y,a_prev, parameters)
    # backward prop.
    gradients, a = rnn_backward(X,Y,parameters, cache)
    
    #clip
    gradients=clip(gradients,5)
    
    #update parameters
    parameters=update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]
    

In [13]:
#test
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

Loss = 79.42910009643815
gradients["dWaa"][1][2] = 5.0
np.argmax(gradients["dWax"]) = 38
gradients["dWya"][1][2] = -0.09062425821345271
gradients["db"][4] = [2.15345883]
gradients["dby"][1] = [0.0937813]
a_last[4] = [-0.93642386]


### model training

In [14]:
def model(data, ix_to_char, char_to_ix, num_iterations=35000, n_a=50, dino_names=7, vocab_size=27):  
    '''
     Trains the model and generates dinosaur names. 
     Returns: parameters -- learned parameters
     '''
    # retrieve dimensions from vocab_size
    n_x, n_y=vocab_size, vocab_size
    
    #initialize parameters
    parameters=initialize_parameters(n_a, n_x, n_y)
    #initialize loss 
    loss=get_initial_loss(vocab_size, dino_names)
    
    # build list of training examples
    with open('dinos.txt') as f:
        examples=f.readlines()
    examples=[x.lower().strip() for x in examples]
    
    # shuffle list of names
    np.random.shuffle(examples)
    
    #initialize the hidden state of lstm
    a_prev=np.zeros([n_a,1])
    
    #optimization loop:
    for i in range(num_iterations):
        #define one training example (X,Y)
        index = i%len(examples)
        X = [None]+[char_to_ix[ch] for ch in examples[index]]
        Y = X[1:]+[char_to_ix['\n']]
    #  one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
    curr_loss, gradients, a_prev = optimize(X,Y,a_prev, parameters, learning_rate=0.01)
    
    loss = smooth(loss, curr_loss)
    
    #check every 2000 iterations:
    if i%2000==0:
        print('iteration: {}, loss: {:.f}'.format(j, loss), '\n', sep='')
        
        for name in range(dino_names):
            
            sample_indices=sample(parameters, char_to_ix)
            print_sample(sample_indices, ix_to_char)
            
        print('\n')
    return parameters


In [15]:
parameters = model(data, ix_to_char, char_to_ix)